In [1]:

import pandas as pd
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import torch
from torch import nn
import matplotlib.pyplot as plt

# импортируем трансформеры
import transformers
from transformers import AdamW
import warnings
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix


from torch.utils.data import DataLoader, TensorDataset, DataLoader, RandomSampler
from sklearn.utils.class_weight import compute_class_weight
from torchmetrics.classification import BinaryAccuracy

#some useful lybs
import random as random

# metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

/home/andrey/Documents/env/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:

class BERT_architecture(nn.Module):
    def __init__(self, bert):     
      super(BERT_architecture, self).__init__()
      self.bert = bert 
      # dropout layer
      self.dropout = nn.Dropout()      
      self.tahn = nn.Tanh()
      self.Sigmoid1 = nn.Sigmoid()
      self.Sigmoid2 = nn.Sigmoid()
      self.fc1 = nn.Linear(312,256)
      self.fc2 = nn.Linear(256,100)
      self.fc3 = nn.Linear(100,32)
      self.fc4 = nn.Linear(32,1)
      #self.fc3 = nn.Linear(64,5)

      self.softmax = nn.LogSoftmax(dim=1)
      
    #define the forward pass
    def forward(self, sent_id, mask):
      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False) 
      cls_hs = nn.functional.normalize(cls_hs)
      #print(f'cls : {cls_hs.shape}')  #cls : torch.Size([64, 312])
      x = self.fc1(cls_hs)
      x = self.Sigmoid1(x)
      x = self.dropout(x)

      x = self.fc2(x)
      x = self.Sigmoid2(x)

      x = self.fc3(x)
      x = self.Sigmoid2(x)
      x = self.fc4(x)
      x = self.Sigmoid2(x)
      return x

In [10]:
local_bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [11]:
model_BERT = BERT_architecture(local_bert)

In [13]:
model_BERT.load_state_dict(torch.load('/home/andrey/Documents/nlp_project_W10D4-5_BERT_group/hosp_feedback_folder/models/model_bert.mod', weights_only=True))


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torch.nn.parallel.data_parallel.DataParallel was not an allowed global by default. Please use `torch.serialization.add_safe_globals([DataParallel])` or the `torch.serialization.safe_globals([DataParallel])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types ac